# Hands-on Lab 7: Controls Part 1
## EECS 16B: Designing Information Devices and Systems II, Fall 2022

Updated by Steven Lu, Kaitlyn Chan (2021)

Updated by Steven Lu (2022)

Updated by Mingyang Wang, Shrey Aeron, and Megan Zeng (2022)

## <span style="color:navy">Table Of Contents</span>
-----
* [Introduction](#intro)
* [Part 1: Open-Loop Control](#part1)
* [Part 2: Closed-Loop Control](#part2)


<a id='intro'></a>
## <span style="color:navy">Introduction</span>

In the previous lab, we developed a linear model for our system to better understand how our motor inputs get "translated" into wheel velocity. In this lab, we will use this model to drive S1XT33N in a straight line. We will do so in 2 different ways: open-loop control and closed-loop control. In the next lab, we will extend the controller to enable controlled turning. 

## Lab Note
* [Lab Note](https://eecs16b.org/lab/notes/lab_note7.pdf)

Review the lab note. The lab note provides a technical review of the concepts covered in lab as well as theoretical connections.


## Pre-Lab

Complete the pre-lab assignment before doing the lab. Submit your answers to the Gradescope assignment "[[Lab] Pre-Lab 7: Controls Part 1](https://www.gradescope.com/courses/414337/assignments/2375507)". Pre-Lab 7 is due on **Monday, October 31 at 11:59pm**. No late submissions will be accepted.


<a id='part1'></a>
# <span style="color:navy">Part 1: Open-Loop Control</span>

## 1.0 Introduction to Open-Loop Control

We will begin by exploring how to control our car using **open-loop control**. Open-loop control refers to a controller that does not take in feedback from the target system; it blindly sets the inputs to what it calculates based on the model.

 For S1XT33N, open loop control will attempt to calculate the inputs $u_L$ and $u_R$ to make both wheels run at the same velocity v* (the operating velocity we calculated in the previous lab). 

**First, fill in your model parameters and target velocity from Lab 6: System ID below.**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import utils
%matplotlib inline

In [ ]:
# YOUR PARAMETERS HERE
theta_left =
theta_right =
beta_left =
beta_right =
v_star =

simulator = utils.Simulator(theta_left, theta_right, beta_left, beta_right)

In [ ]:
params = np.array([(theta_left, theta_right),
                   (beta_left, beta_right)])
d0 = (0,  0)
sim_length = 10 # sim length
mismatch_error = 0.1 # 10% model mismatch

<a id='part1.1'></a>
## 1.1 Open-Loop Controller Design

Use the linear model parameters you calculated in the previous lab to design your open-loop controller. Your controller takes in the desired wheel velocity, $v_{star}$, and returns the motor inputs, $u_L$ and $u_R$, to set the model wheel velocities to $v_{star}$.

Tip: Since we defined your parameters $\theta_L$, $\theta_R$, etc in a code block above, you can use those same variables within your open-loop functions.

In [ ]:
# DESIGN YOUR OPEN-LOOP CONTROL HERE
def drive_straight_left_ol(v_star):
    """Returns the open loop input 'u' for the left wheel"""
    # YOUR CODE HERE
    raise NotImplementedError() # REMOVE THIS LINE
    return ...

def drive_straight_right_ol(v_star):
    """Returns the open loop input 'u' for the right wheel"""
    # YOUR CODE HERE
    raise NotImplementedError() # REMOVE THIS LINE
    return ...

<a id='part1.2'></a>
## 1.2 Open-Loop Simulation


Now, let's do a quick simulation to check the behavior of the car based on our functions above.

### Mismatch
 There could be a mismatch between the estimated model parameters $\beta_{L,R}, \theta_{L,R}$ and real model parameters $\beta^*_{L,R}, \theta^*_{L,R}.$ In this simulation, we assumed that the mismatch error is the same for both $\beta_{L,R}$ and $\theta_{L,R}$


$$\theta^*_{L,R}=\theta_{L,R} \times (1+\text{mismatch\_error})$$
$$\beta^*_{L,R}=\beta_{L,R} \times (1+\text{mismatch\_error})$$


In the left two plots, we assume the physical system perfectly matches our model. On the right, we assume the physical system is 10% different from our model, meaning the model parameters of the wheels have a 10% mismatch. Don't worry too much about the $\delta$ (delta) plot for now; we will come back to what this is later on in the lab.

The 5th plot titled "Trajectory of the car" shows how the car would travel depending on which model is used. In the perfect model, the car is driving straight. In the case with model mismatch, the car continues to spin right.


In [ ]:
titles = ["Open-loop control with perfect model",
          "Open-loop control with model mismatch"]
_ = utils.two_sims(titles, simulator, v_star, drive_straight_left_ol, drive_straight_right_ol)
utils.plot_car_traj(titles[1], simulator, v_star, drive_straight_left_ol, drive_straight_right_ol,mismatch_error=0.1,sim_length=sim_length)

<a id='part1.3'></a>
## 1.3 Jolt Calculation

When starting our motors from rest, we need to apply a large initial PWM to overcome static friction and "jolt" the motors into motion. However, as each side responds differently to the input; doing so could cause the car to immediately turn at the start and veer off course. 

Thus, we need to calculate jolt values that result in the two wheels attaining the same velocity, which we will choose to be the maximum velocity attainable by both wheels. This is equivalent to min(max(v_left), max(v_right)).

- **Copy your `data_coarse.txt` file from Lab 6 into the same directory as this notebook.**
- Run the code below to find and print your initial jolt values.
- These jolt values should be the input values that correspond to the points where the blue and yellow lines intersect the green line respectively.

In [ ]:
left_jolt, right_jolt = utils.find_jolt()
print('int left_jolt = {};\nint right_jolt = {};'.format(left_jolt, right_jolt))

<a id='part1.4'></a>
## 1.4 Open-Loop Implementation

Now we will implement the controller on the Arduino. The sketch **`open_loop.ino`** contains some code that implements open-loop control.

- **If you've changed any of the pins from their default values in System ID, make sure those changes are reflected in the pin definitions.**

- Fill in these values in `CODE BLOCK CON1`:
    - estimated parameters: `theta_left`, `theta_right`, `beta_left`, `beta_right`
    - desired velocity: `v_star` 
    - initial jolt values: `left_jolt`, `right_jolt`
- Additionally, we need to compute the PWM input to the motors in `CODE BLOCK CON2`:
    - Complete `driveStraight_left()` and `driveStraight_right()` to compute `left_cur_pwm` and `right_cur_pwm`
    - This is similar to the `drive_straight_left_ol` and `drive_straight_right_ol` functions you wrote above, but now using C/C++ syntax instead of Python.
    - Tip: Your $\theta$ and $\beta$ values are defined globally at the top of the .ino file, so you can reference them inside your function.
- Fill out `CODE BLOCK CON3` to calculate the control inputs using the functions from `CODE BLOCK CON2`.

- Upload the sketch to your Arduino. Suspend your car over your desk and verify that your motors work as expected.

Once you are satisfied that your open loop control is implemented, run your car on the ground. 

Most likely, S1XT33N will not drive in a straight line: you will see that it is still turning quite a bit. In the next section, we will use closed-loop control to use feedback to drive in a straight line.

-----

<a id='part2'></a>
# <span style="color:navy">Part 2: Closed-Loop Control</span>

<a id='part20'></a>
## 2.0 Introduction to Closed-Loop Control

To correct for the limitations of open-loop control, we need a way to detect that the car is turning. Fortunately, we already have encoders that can measure the distance traveled by each wheel. We can take this information and "feed it back" to the Arduino as an input so that it can correct the trajectory. This is the basis for a closed-loop controller.


<a id='part21'></a>
## 2.1 Closed-Loop Design

We define $\delta[i] = d_L[i] - d_R[i]$, the difference between the distances traveled by the left and right wheels.
Our closed-loop controller takes the $\delta$ value and adjusts the left and right inputs to correct for turns or disturbances. We define feedback factors $f_{left}$ and $f_{right}$ to control the amount of correction of each wheel. 

In Parts 2.1-2.4, the goal of our closed-loop controller is to converge S1XT33N's trajectory to a straight line in a reasonable amount of time. It is okay if the car starts off turning and then drives in a straight line; we will correct for this in Part 2.5.

Using the model derived in the lab note, we can begin implementing the closed-loop controller.

Start by running the code below to set up the simulator and fill in `drive_straight_left_cl` and `drive_straight_right_cl`, which compute the wheel inputs using the desired velocity `v_star`, error term `delta`, and feedback factors `f_left` and `f_right` (which will be defined in the next section).

In [ ]:
simulator = utils.Simulator(theta_left, theta_right, beta_left, beta_right)

In [ ]:
# DESIGN YOUR CLOSED-LOOP CONTROL HERE USING f_left AND f_right AS YOUR FEEDBACK TERMS
def drive_straight_left_cl(v_star, delta):
    """Returns the closed loop input 'u' for the left wheel"""
    # YOUR CODE HERE
    raise NotImplementedError() # REMOVE THIS LINE
    return ...

def drive_straight_right_cl(v_star, delta):
    """Returns the closed loop input 'u' for the right wheel"""
    # YOUR CODE HERE
    raise NotImplementedError() # REMOVE THIS LINE
    return ...


<a id='part22'></a>
## 2.2 Closed-Loop Simulation and f-Value Selection

With the input-calculating functions implemented, we will choose feedback factor values to stabilize the $\delta[i]$ update equation. There should be a range of parameter values which (theoretically) make it stable, so we will tune the gains to achieve the desired performance.

Take a look at each pair of perfect/mismatched-model plots below for each f-value pair and relate how our choice in f-values affects the car's performance. For each pair of plots, ask yourself these questions:
1. Does the car seem to drive straight in the perfectly matched model?
2. Does the car seem to correct for any errors that arise from a mismatched model? Does it end up driving straight?
3. How well does the car correct for errors?
3. Why do the chosen f-values make our system exhibit such behaviors?

Try the 5 different sets of gains below and see the effect of each.

In [ ]:
# Maps description of f_values to (f_left, f_right)
f_values = {
    "marginally stable": (-0.3, 0.3),
    "stable, corrects error in one step": (0.5, 0.5),
    "oscillatory, marginally stable": (1, 1),
    "oscillatory, unstable": (1.1, 1.1),
    "stable": (0.2, 0.2),
}

for key in f_values:
    f_left, f_right = f_values[key]

    titles = ["Closed-loop control with perfect model",
              "Closed-loop control with model mismatch, fL={}, fR={}".format(f_left, f_right)]
    _, delta = utils.two_sims(titles, simulator, v_star, drive_straight_left_cl, drive_straight_right_cl)

    print("fL={}, fR={}".format(f_left, f_right))
    print("Eigenvalue of system: 1-fL-fR={:.2g}; {}".format(1-f_left-f_right, key))
    plt.show()
    steady_state_error = delta[-1]

Now that you've taken a look at the behavior of various values for $f_{left}$ and $f_{right}$, choose your own for your car model! Fill in your chosen values in the code below to see how your model behaves. Does it do what you expected it to?

In [ ]:
f_left, f_right = None, None # TODO: Fill in your chosen f values

titles = ["Closed-loop control with perfect model",
          "Closed-loop control with model mismatch, fL={}, fR={}".format(f_left, f_right)]
_, delta = utils.two_sims(titles, simulator, v_star, drive_straight_left_cl, drive_straight_right_cl)

print("fL={}, fR={}".format(f_left, f_right))
print("Eigenvalue of system: 1-fL-fR={:.2g}; {}".format(1-f_left-f_right, key))
plt.show()
steady_state_error = delta[-1]

<a id='part23'></a>
## 2.3 Closed-Loop Implementation

Now implement the controller on the Arduino. The sketch **`closed_loop.ino`** contains some code that implements the closed loop control. 

- If you've changed any of the pins from their default values, make sure those changes are reflected in the pin definitions.
- Fill out `CODE BLOCK CON1` with your control gains **`f_left`** and **`f_right`** as well as your $\theta$, $\beta$, and $v_{star}$ values from System ID.
    - Finding the values that work best for your car is an iterative process, and you'll have to tune these values.
- Fill out `CODE BLOCK CON2` with your closed-loop control scheme.
- Fill out `CODE BLOCK CON4` in the body of the loop function to calculate the control inputs. Inspect this code to see how the control system is implemented. 
    - Ignore the term `delta_ss` for now. We will implement this in a bit.
- Upload the sketch to your Arduino. Suspend your car over your desk and verify that your motors work as expected.
    - A good sanity check is to manually hold one wheel in place; the controller should cause the other wheel to stop as well.

Once you are satisfied that your closed loop control is implemented, run your car on the ground. 

-----

<a id='part24'></a>
## 2.4 f-Value Tuning

How did the car behave? It probably didn't go perfectly straight with the first set of f-values that you picked. To find the optimal f values, we will analyze the behavior of our car. For our system, we want $\delta[i]$ (the blue line) to converge to 0 in a reasonable amount of time.

small f-values | large f-values
- | - 
![alt](images/small_f.png) | ![alt](images/large_f.png)
 
The above plots can be summarized as:
- If $f_{left}$ and $f_{right}$ are too small, convergence takes a long time.
- If $f_{left}$ and $f_{right}$ are too big, the system oscillates.
We can use this information to select f-values for straight motion.

With this in mind, try different f-values and run the car on the ground again, observing its trajectory. This is an iterative process and you will likely have to try several sets of f-values until you find one that works well for your particular car. 

<a id='part25'></a>

## 2.5 Steady State Error Correction

Oftentimes, delta will converge to some non-zero constant rather than zero. This means that the car eventually converges to driving straight, but with a different bearing (i.e. the car turned initially but then drives straight). However, we want the car to travel in (approximately) the same direction as when it started moving. 

We can accomplish this by manually adding the known steady state error $\delta_{ss}$ to each calculation of the error $\delta$.
We define our new error term as $\delta[i]=d_L - d_R + \delta_{ss}$.


In [ ]:
plt.figure(figsize=(5, 7))
plt.subplot(211)
# Simulate using steady_state_error
d = simulator.simulate(
    v_star, drive_straight_left_cl, drive_straight_right_cl, mismatch_error=0.1, sim_length=20, offset=steady_state_error)
delta = simulator.plot(d)
plt.title("Closed-loop control with model mismatch and\nsteady state error correction, fL={}, fR={}"
          .format(f_left, f_right))
plt.subplot(212)
plt.plot(delta, 'r')
plt.ylabel('delta')
plt.xlabel('n(sample)')
plt.grid()

`closed_loop.ino` collects data as it runs, keeping track of $\delta[i]$ over time. Like `dynamics_data.ino` from the System ID lab, data will be stored in local memory as long as power to the Arduino is not cut off. After you have finished tuning your f-values, do the following:
- Upload the `closed_loop.ino` program again and let the car run until it settles to (approximately) straight; it should do so by the time it naturally comes to a stop.
- With the Arduino still powered by `Vin`, plug in the USB and open the Serial monitor. Type any character into the input box and hit enter. Your data will be printed into the Serial Monitor.
- Note the values of $\delta$.
    - **If $\delta$ doesn't converge (reaches steady state), the car is not going straight according to the sensors.**
        - It's okay if it isn't exactly constant (ie $\pm 1$)
    - The value that the $\delta$ converges to is the $\delta_{ss}$ value.

## Implementing Steady State Error Correction

- Input your value of $\delta_{ss}$ in `CODE BLOCK CON3` in **`closed_loop.ino`**.
- Make sure you understand why you *add* this value in `CODE BLOCK CON4`.
    - For example, if the car drives straight but turned right from its initial heading, we want the right wheel to travel farther so it turns back towards the left. Adding a positive number to delta accomplishes this.
- Reupload the `closed_loop.ino` program, unplug the Arduino from the computer, and run your car on the ground to observe its behavior with steady state error correction.

<img width='30px' align='left' src="http://inst.eecs.berkeley.edu/~ee16b/sp16/lab_pics/check.png">

## <span style="color:green">CHECKOFF</span>

- **Have all questions, code, and plots completed in this notebook.**

- <span style="color:green"> **Demonstrate your car going straight.** Here is our definition of “straight” based on the floor tiling your car has to travel within at various locations inside and outside the Cory 125 lab: </span>
   - Outside Cory 125 double doors: 3x11 on black tiles
   - Side entrance hallway, from the pink to red lines: 2x7 tiles

- **Be prepared to answer conceptual questions about the lab.**

**After checkoff:**

- <span style="color:#ff0000"> **DO NOT DISMANTLE ANY CIRCUITS FROM YOUR PROJECT BREADBOARD! You will need them for the rest of the project!**</span>

- Make sure that all electrical components that you received in your lab kit are stored neatly and safely for use in future labs.

- **Remove your battery caps from your batteries to avoid accidental short circuits that might drain their voltage.**

- **SAVE ALL YOUR DATA!!** Data stored on the lab computers often gets deleted automatically. Please store it on your personal flash drive or cloud storage like Google Drive, and not on the lab computers! If you used DataHub, it should save through your CalNet ID. **You will still need to back up your `.ino` files**

- **Clean up your lab bench areas!** This includes turning off all lab equipment, putting all lab equipment probes back on the racks, returning all components you borrowed, putting the rest of the components back in your lab kits neatly, and throwing away any stripped wires.
